In [1]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

%matplotlib inline

# Composition Analysis

The objective of this analysis is to determine whether composition impacts stability (whether compounds with similar composition have similar stability). To do this, I mainly focused on compounds with a single A compound to remove confounding effects from other A site compounds and then do the same for the B site. 
- From the analysis, the conclusion is:
    - It is possible that including composition features may help the performance:
        - We can model the features by:
        1. Use the features of:
            - 'Ba', 'Pr', 'Y', 'La', 'Sr', 'Other' on the A-site
            - 'Fe', 'V', 'Ni', 'Mn', 'Other' on the B-site
        2. Use the features separated by classification and regression (to use less feature variables):
            - Classification:
                - 'La', 'Pr', 'Y', 'Other' on the A-site
                - 'V', 'Other' on the B-site
            - Regression:
                - 'Sr', 'Ba', 'Other' on the A-site
                - 'Ni', 'Fe', 'V', 'Other' on the B-site
        3. For classification, we may even want to include less features that captures interaction effects:
            - Indicator feature for whether A-site includes 'Pr', 'La', 'Y'
            - Another indicator feature for when A-site includes 'Pr' or 'Y' and B-site includes 'V'

    - It is possible that the above features may not help when combined with other numerical features

    - Separately, based on the performance breakdown shown by Jiaying in last meeting, I think it makes sense to stratify train test samples based on major A-site / B-site composition combinations, so that the performance is fairly measured (for ex, if model does particularly well for 'Ba' and test set contains higher proportion of 'Ba' than train, then the evaluation is probably not 'fair'

In [2]:
FILE_PATH = '../../data/stability_paper_data/augmented sample.xlsx'

In [3]:
df = pd.read_excel(FILE_PATH, sheet_name='Sheet1')

In [4]:
df

,index,Material Composition,goldschmidt_TF,goldschmidt_TF_ionic,octahedral_factor,octahedral_factor_ionic,A_O,B_O,A_B,num_of_atoms_host_Asite0,...,Bsite_NdValence_range,Bsite_NfUnfilled_range,Bsite_NfValence_range,Bsite_NpUnfilled_range,Bsite_NpValence_range,Bsite_NsUnfilled_range,Bsite_NsValence_range,Bsite_NUnfilled_range,EnergyAboveHull,Formation_energy
0,1,Ba1Sr7V8O24,1.021823,0.976828,0.414286,0.385714,2.86125,1.98000,2.04125,1,...,0,0,0,0,0,0,0,0,29.747707,-2.113335
1,2,Ba2Bi2Pr4Co8O24,0.987385,0.889057,0.378571,0.464286,2.69500,1.93000,1.82500,2,...,0,0,0,0,0,0,0,0,106.702335,-1.311863
2,3,Ba2Ca6Fe8O24,0.976009,0.908360,0.452857,0.392857,2.80750,2.03400,2.04150,2,...,0,0,0,0,0,0,0,0,171.608093,-1.435607
3,4,Ba2Cd2Pr4Ni8O24,1.026809,0.865275,0.342857,0.492857,2.73000,1.88000,1.81000,2,...,0,0,0,0,0,0,0,0,284.898190,-0.868639
4,5,Ba2Dy6Fe8O24,0.909001,0.916519,0.452857,0.392857,2.61475,2.03400,1.84875,2,...,0,0,0,0,0,0,0,0,270.007913,-1.746806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,2134,Sr8Nb1Co7O24,1.030498,0.918074,0.391964,0.463393,2.84000,1.94875,1.98875,8,...,3,0,0,0,0,1,1,4,54.915236,-1.449677
2134,2135,Sr8Nb2Co6O24,1.020678,0.918634,0.405357,0.462500,2.84000,1.96750,2.00750,8,...,3,0,0,0,0,1,1,4,33.280498,-1.651437
2135,2136,Sr8Nb4Co4O24,1.001588,0.919757,0.432143,0.460714,2.84000,2.00500,2.04500,8,...,3,0,0,0,0,1,1,4,0.000000,-2.048177
2136,2137,Sr8Nb6Co2O24,0.983199,0.920883,0.458929,0.458929,2.84000,2.04250,2.08250,8,...,3,0,0,0,0,1,1,4,30.771125,-2.314135


In [5]:
A_SITE_COLUMNS = [
    'A site #1',
    'A site #2',
    'A site #3',
]

B_SITE_COLUMNS = [
    'B site #1',
    'B site #2',
    'B site #3',
]

TARGET_COLUMN = 'energy_above_hull (meV/atom)'

In [6]:
orig_composition_df = pd.read_excel(
    '../../data/stability_paper_data/full_features.xlsx', sheet_name='DFT Calculated Dataset')

In [31]:
a_site_elements = orig_composition_df[A_SITE_COLUMNS].fillna('null').values.flatten()
a_site_elements = np.unique(a_site_elements[a_site_elements != 'null'])

b_site_elements = orig_composition_df[B_SITE_COLUMNS].fillna('null').values.flatten()
b_site_elements = np.unique(b_site_elements[b_site_elements != 'null'])

In [41]:
temp = orig_composition_df[A_SITE_COLUMNS]

In [44]:
temp.shape

(1929, 3)

In [45]:
1929 * 3

5787

In [47]:
pd.melt(temp)['value'].value_counts()

Ba    598
Sr    466
Pr    440
La    422
Y     390
Ca    279
Zn    160
Sm     64
Gd     63
Dy     62
Ho     62
Nd     61
Bi     15
Cd     15
Sn     15
Mg      8
Ce      3
Er      1
Name: value, dtype: int64

In [34]:
np.intersect1d(a_site_elements, b_site_elements)

array(['Mg', 'Sn', 'Y', 'Zn'], dtype=object)

In [37]:
orig_composition_df[orig_composition_df['B site #1'] == 'Y']

,Material Composition,A site #1,A site #2,A site #3,B site #1,B site #2,B site #3,X site,Number of elements,energy_above_hull (meV/atom),formation_energy (eV/atom)
437,Ba8Y1Fe7O24,Ba,NaN,NaN,Y,Fe,NaN,O,4,73.802314,-1.554716
438,Ba8Y2Fe6O24,Ba,NaN,NaN,Y,Fe,NaN,O,4,63.151297,-1.691099
439,Ba8Y4Fe4O24,Ba,NaN,NaN,Y,Fe,NaN,O,4,64.504961,-1.941074
440,Ba8Y8O24,Ba,NaN,NaN,Y,NaN,NaN,O,3,141.664751,-2.338811
1665,Sr8Y1V7O24,Sr,NaN,NaN,Y,V,NaN,O,4,64.140924,-2.172679
1666,Sr8Y2V6O24,Sr,NaN,NaN,Y,V,NaN,O,4,93.842303,-2.231961
1667,Sr8Y4V4O24,Sr,NaN,NaN,Y,V,NaN,O,4,112.174952,-2.378805


In [32]:
a_site_elements

array(['Ba', 'Bi', 'Ca', 'Cd', 'Ce', 'Dy', 'Er', 'Gd', 'Ho', 'La', 'Mg',
       'Nd', 'Pr', 'Sm', 'Sn', 'Sr', 'Y', 'Zn'], dtype=object)

In [33]:
b_site_elements

array(['Al', 'Co', 'Cr', 'Cu', 'Fe', 'Ga', 'Ge', 'Hf', 'Ir', 'Mg', 'Mn',
       'Mo', 'Nb', 'Ni', 'Os', 'Pd', 'Pt', 'Re', 'Rh', 'Ru', 'Sc', 'Si',
       'Sn', 'Ta', 'Tc', 'Ti', 'V', 'W', 'Y', 'Zn', 'Zr'], dtype=object)

In [ ]:
composition_df = pd.read_excel(FILE_PATH, sheet_name='Sheet1')
composition_df['y_clf'] = (composition_df['energy_above_hull (meV/atom)'] <= 40).astype(int)

In [ ]:
# a1_df are those with single A site
# b1_df are those with single B site

a1_mask = composition_df[A_SITE_COLUMNS].isna().sum(axis=1) == 2
b1_mask = composition_df[B_SITE_COLUMNS].isna().sum(axis=1) == 2

a1_df = composition_df[a1_mask].copy()
b1_df = composition_df[b1_mask].copy()

In [ ]:
a1_df['A site #1'].value_counts(normalize=True).round(3).head(10)

In [ ]:
b1_df['B site #1'].value_counts(normalize=True).round(3).head(10)

In [ ]:
top_asite_compounds = a1_df['A site #1'].value_counts().head(5).index.values
sub_df_a = a1_df[a1_df['A site #1'].isin(top_asite_compounds)]

top_bsite_compounds = b1_df['B site #1'].value_counts().head(5).index.values
sub_df_b = b1_df[b1_df['B site #1'].isin(top_bsite_compounds)]

In [ ]:
print('Top A Site compounds for study:', top_asite_compounds)
print('Top B Site compounds for study:', top_bsite_compounds)

## 1. Conclusion from Stats
- A site compounds with La, Pr, Y, are more likely to have stable behavior than Ba, Sr 
- B site compounds with V are more likely to have stable behavior than others
- B site compounds with Ni, Fe tend to have particularly unstable behavior
- Judging from the F-stat p-value at 5% significance (to see if each compound is a significant separator among those that contain the compound and those that don't contain), I would separate those as:
    - 'Ba', 'Pr', 'Y', 'La', 'Sr', 'Other' on the A-site
    - 'Fe', 'V', 'Ni', 'Mn', 'Other' on the B-site

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

sns.boxplot(data=sub_df_a, 
            x='A site #1', 
            y='energy_above_hull (meV/atom)',
            ax=ax)

plt.show()

In [ ]:
sub_df_a.groupby('A site #1')[TARGET_COLUMN].agg(['mean', 'std']).round(1)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

sns.boxplot(data=sub_df_b, 
            x='B site #1', 
            y='energy_above_hull (meV/atom)',
            ax=ax)

plt.show()

In [ ]:
sub_df_b.groupby('B site #1')[TARGET_COLUMN].agg(['mean', 'std']).round(1)

In [ ]:
f_stat_result = []

for compound in top_asite_compounds:
    group1 = sub_df_a[sub_df_a['A site #1'] == compound][TARGET_COLUMN].values
    group2 = sub_df_a[sub_df_a['A site #1'] != compound][TARGET_COLUMN].values
    
    anova_result = stats.f_oneway(group1, group2)
    
    f_stat_result.append({'compound': compound, 'pval': anova_result.pvalue})
    
f_stat_df = pd.DataFrame(f_stat_result).sort_values('pval')

In [ ]:
f_stat_df.round(8)

In [ ]:
f_stat_result = []

for compound in top_bsite_compounds:
    group1 = sub_df_b[sub_df_b['B site #1'] == compound][TARGET_COLUMN].values
    group2 = sub_df_b[sub_df_b['B site #1'] != compound][TARGET_COLUMN].values
    
    anova_result = stats.f_oneway(group1, group2)
    
    f_stat_result.append({'compound': compound, 'pval': anova_result.pvalue})
    
f_stat_df = pd.DataFrame(f_stat_result).sort_values('pval')

In [ ]:
f_stat_df.round(8)

## 2. Decision Trees to See Which Features Give the Greatest Separation in Predicting the Target
- Features that give better classification are different for Regression vs. Classification
    - Classification
        - 'La', 'Pr', 'Y' are particularly important features from A site to determine whether stable or not
        - 'V', 'Other' are particularly important features from B site to determine whether stable or not
    - Regression
        - 'Other', 'Sr', 'Ba' are particularly important features from A site for regression
        - 'Ni', 'Fe', 'V' are particularly important features from B site for regression

In [ ]:
from sklearn.tree import (
    DecisionTreeRegressor, DecisionTreeClassifier,
    plot_tree
)

In [ ]:
A_SITE_COMPOUND_MAP = {
    'Ba': 'Ba', 
    'Pr': 'Pr', 
    'Y': 'Y', 
    'La': 'La', 
    'Sr': 'Sr', 
}

B_SITE_COMPOUND_MAP = {
    'Fe': 'Fe', 
    'V': 'V', 
    'Ni': 'Ni', 
    'Mn': 'Mn',
}

In [ ]:
def get_feature_importance(model, feature_names):
    importance_df = pd.DataFrame({'feature': feature_names,
                                  'importance': model.feature_importances_,}).sort_values(
        'importance', ascending=False).round(2)
    return importance_df

In [ ]:
a1_df_copy = a1_df.copy()
a1_df_copy['A site #1'] = a1_df_copy['A site #1'].map(A_SITE_COMPOUND_MAP).fillna('Other')
a1_X = a1_df_copy['A site #1']
a1_y = a1_df_copy[TARGET_COLUMN]
a1_y_clf = a1_df_copy['y_clf']
a1_X = pd.get_dummies(a1_X, drop_first=False)

In [ ]:
b1_df_copy = b1_df.copy()
b1_df_copy['B site #1'] = b1_df_copy['B site #1'].map(B_SITE_COMPOUND_MAP).fillna('Other')
b1_X = b1_df_copy['B site #1']
b1_y = b1_df_copy[TARGET_COLUMN]
b1_y_clf = b1_df_copy['y_clf']
b1_X = pd.get_dummies(b1_X, drop_first=False)

In [ ]:
clf = DecisionTreeClassifier(max_depth=5,
                             min_samples_split=5,
                             min_samples_leaf=5)

clf.fit(a1_X, a1_y_clf)

fig, ax = plt.subplots(figsize=(8, 8))

plot_tree(clf, 
          filled=True, 
          feature_names=a1_X.columns,
          ax=ax)
plt.title('Decision Tree Classifier for A Site')
plt.show()

In [ ]:
get_feature_importance(clf, a1_X.columns)

In [ ]:
reg = DecisionTreeRegressor(max_depth=5,
                            min_samples_split=5,
                            min_samples_leaf=5)

reg.fit(a1_X, a1_y)

fig, ax = plt.subplots(figsize=(8, 8))

plot_tree(reg, 
          filled=True, 
          feature_names=a1_X.columns,
          ax=ax)
plt.title('Decision Tree Regressor for A Site')
plt.show()

In [ ]:
get_feature_importance(reg, a1_X.columns)

In [ ]:
clf = DecisionTreeClassifier(max_depth=5,
                             min_samples_split=5,
                             min_samples_leaf=5)

clf.fit(b1_X, b1_y_clf)

fig, ax = plt.subplots(figsize=(8, 8))

plot_tree(clf, 
          filled=True, 
          feature_names=b1_X.columns,
          ax=ax)
plt.title('Decision Tree Classifier for B Site')
plt.show()

In [ ]:
get_feature_importance(clf, b1_X.columns)

In [ ]:
reg = DecisionTreeRegressor(max_depth=5,
                            min_samples_split=5,
                            min_samples_leaf=5)

reg.fit(b1_X, b1_y)

fig, ax = plt.subplots(figsize=(8, 8))

plot_tree(reg, 
          filled=True, 
          feature_names=b1_X.columns,
          ax=ax)
plt.title('Decision Tree Regressor for B Site')
plt.show()

In [ ]:
get_feature_importance(reg, b1_X.columns)

## 3. Interaction Effects Between Major A site & B site Compounds
- From the box plot, we can see that:
    - When A site doesn't include the compounds 'Pr', 'La', 'Y' then it is likely to be unstable
    - When A site includes the compounds 'Pr' or 'Y' and B site includes the compound 'V' then it is likely to be stable

In [ ]:
A_CLF_COMPOUNDS = ['La', 'Pr', 'Y']
B_CLF_COMPOUNDS = ['V']

In [ ]:
def get_compound_feature(row, 
                         compound_cols=['A site #1', 'A site #2', 'A site #3'],
                         ref_compounds=A_CLF_COMPOUNDS):
    
    included_compounds = []
    
    for col in compound_cols:
        included_compounds.append(row[col])
        
    features = list(set(included_compounds) & set(ref_compounds))
    
    if not len(features):
        return 'Other'
    else:
        return features[0]

In [ ]:
composition_df['asite_feature'] = composition_df.apply(lambda r: 
                                                       get_compound_feature(r,
                                                                            compound_cols=['A site #1', 'A site #2', 'A site #3'],
                                                                            ref_compounds=A_CLF_COMPOUNDS),
                                                        axis=1)
composition_df['bsite_feature'] = composition_df.apply(lambda r: 
                                                       get_compound_feature(r,
                                                                            compound_cols=['B site #1', 'B site #2', 'B site #3'],
                                                                            ref_compounds=B_CLF_COMPOUNDS),
                                                       axis=1)

In [ ]:
composition_df['asite_feature'].value_counts()

In [ ]:
composition_df['bsite_feature'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

sns.boxplot(data=composition_df, 
            x='asite_feature', 
            hue='bsite_feature',
            y='energy_above_hull (meV/atom)',
            ax=ax)

ax.axhline(y=40, c='r', linewidth=3)

plt.show()